In [1]:
import Thymio as th
import time
import costmap_path as cp
import cv2
import numpy as np
import threading
import vision.detect_aruco as da

#FIXME: not right original image
original_image = cv2.imread('grid.png',0)
costmap, block_size, start, goal, display_image=cp.init(original_image)
path,costmap = cp.update(costmap, block_size,start, goal, display_image,[])
next_goal = path[0]
current_pos = np.array([start[0],start[1],0])
current_speed = np.array([0,0,0])
camera_position = (0,0,0)
z_predicted = np.concatenate([current_pos,current_speed]) #kalman_state
sigma_predicted = np.diag([0.1,0.1,0.1,0,0,0])
print("Open thymio")
thymio = th.Thymio()
await thymio.iniateLock()
print("Thymio opened")
camera_on = True
vars_towait = ['prox.horizontal','motor.right.speed','motor.left.speed','motor.right.target','motor.right.target']
aruco_thread = threading.Thread(target=da.main(), daemon=True)
aruco_thread.start()
while True:
    
    thymio.wait_for_variables([])
    next_goal = path[0]
    print(f'Next goal : {next_goal}')
    print(f'Current position : {current_pos}')
    thymio.navigate(current_pos, next_goal)
    current_pos = z_predicted[:3] # Is used for the controller
    it_vars = thymio.get_multiple_variables(["motor.left.speed","motor.right.speed"])
    
    z_filtered, sigma_filtered= thymio.filtering_step(z_k_k_1 = z_predicted, 
                                    sigma_k_k_1 = sigma_predicted, 
                                    V_left_measure =it_vars["motor.left.speed"]*thymio.speedConversion, 
                                    V_right_measure = it_vars["motor.right.speed"]*thymio.speedConversion, 
                                    camera_working=camera_on, 
                                    x_measured=camera_position[0], 
                                    y_measured=camera_position[1],
                                    theta_measured= camera_position[2])
    
    z_predicted, sigma_predicted = thymio.prediction_step(z_filtered, sigma_filtered)

    obstacle,detected = thymio.getObstaclePosition()
    if detected:
        print("Obstacle detected")
        #TODO: use the obstacle position to update the costmap
        path,costmap = cp.update(costmap, current_pos, goal, display_image)
        
    if thymio.robot_close_waypoint(current_pos[:2], next_goal[0], next_goal[1]):
        path.pop(0)
        if len(path) == 0:
            print("Goal reached")
            thymio.stop()
            aruco_thread.join()
            break
        
        
    
    
    
    
    
    
    
    
 







Invalid input! Please enter integers.


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
thymio.set_multiple_variables({"motor.left.target":[0],"motor.right.target":[0]})